In [193]:
import pandas as pd
import geopandas as gpd

from dotenv import load_dotenv
import os

In [192]:
load_dotenv()  # Carga el archivo .env
# Ahora puedes acceder a las variables
ruta_unidad = os.getenv('RUTA_UNIDAD_LOCAL_DRIVE')
ruta_drive = os.getenv('RUTA_LOCAL_DRIVE')
print(ruta_unidad + ruta_drive)

G:\\OneDrive - Ingenio Azucarero Guabira S.A\_DATOS_PYTHON


In [82]:
path_grupos = ruta_unidad + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\DATA\GRUPO_COSECHA.xlsx'
path_catastro = ruta_unidad + r'\OneDrive - Ingenio Azucarero Guabira S.A\INFORMACION GENERAL\CATASTRO\NUEVO\catastro_S13_SABADO.shp'
path_variedades = ruta_unidad + r"\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\DELTA_X\_DETALLE_VARIEDAD.xlsx"

output_path = ruta_unidad + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\DELTA_X'

In [5]:
def crear_xlsx_caneros():
    df_grupos = pd.read_excel(path_grupos, sheet_name='CODIGOS')
    # Seleccionar solo las columnas necesarias
    df_grupos = df_grupos[['CODIGO CAÑERO', 'NOMBRE CAÑERO', 'INSTITUCION']]
    # Renombrar las columnas
    df_grupos = df_grupos.rename(columns={
        'CODIGO CAÑERO': 'COD_CANERO',
        'NOMBRE CAÑERO': 'NOMBRE',
        'INSTITUCION': 'COD_INSTIT'
    })
    # Reordenar las columnas
    df_grupos = df_grupos[['COD_CANERO', 'COD_INSTIT', 'NOMBRE']]
    print(len(df_grupos), 'Cañeros registrados')
    archivo_salida = os.path.join(output_path, 'CANEROS.xlsx')
    df_grupos.to_excel(archivo_salida, index=False)
    print('Se exporto CANHEROS.xlsx exitosamente')

In [7]:
def crear_xlsx_propiedad():
    gdf_catastro = gpd.read_file(path_catastro)
    gdf_cat_filtrado = gdf_catastro[
        (gdf_catastro['obs'] == 'ok') &
        (gdf_catastro['cultivo'].isin(['canha', 'siembra']))]
    print('Area Total de Catastro:', gdf_cat_filtrado['area'].sum(), 'ha. para PROPIEDAD.xlsx')
    df_pivot = gdf_cat_filtrado.groupby(['unidad_01', 'unidad_02'])['area'].sum().reset_index()
    df_pivot['unidad_01'] = df_pivot['unidad_01'].astype(int)
    df_repetidos = df_pivot[df_pivot.duplicated(subset='unidad_01', keep=False)]
    print('Cantidad de repetidos:', len(df_repetidos))
    df_pivot = df_pivot.rename(columns={
        'unidad_01': 'COD_PROPIE',
        'unidad_02': 'NOMBRE_PRO',
        'area': 'AREA_TOTAL'
    })
    # Definir nombre de archivo
    archivo_salida = os.path.join(output_path, 'PROPIEDAD.xlsx')
    # Exportar a Excel
    df_pivot.to_excel(archivo_salida, index=False)
    print('Se exporto PROPIEDAD.xlsx exitosamente')

In [9]:
def crear_xlsx_caneho_propiedad():
    gdf_catastro = gpd.read_file(path_catastro)
    gdf_cat_filtrado = gdf_catastro[
        (gdf_catastro['obs'] == 'ok') &
        (gdf_catastro['cultivo'].isin(['canha', 'siembra']))]
    print('Area Total de Catastro:', gdf_cat_filtrado['area'].sum(), 'ha. para relacion CANERO-PROPIEDAD')
    # Asegurar que estamos trabajando sobre una copia, no una vista
    df_cat_filtrado = gdf_cat_filtrado[['unidad_03', 'unidad_01']].copy()
    # Convertir a entero
    df_cat_filtrado['unidad_03'] = df_cat_filtrado['unidad_03'].astype(int)
    df_cat_filtrado['unidad_01'] = df_cat_filtrado['unidad_01'].astype(int)
    df_grupos = pd.read_excel(path_grupos, sheet_name='CODIGOS')
    # Asegurarse de que ambas columnas estén en tipo entero
    df_grupos['CODIGO CAÑERO'] = df_grupos['CODIGO CAÑERO'].astype(int)
    # Realizar el merge (left join para mantener solo los registros del shapefile filtrado)
    df_combinado = df_cat_filtrado.merge(
        df_grupos,
        left_on='unidad_03',
        right_on='CODIGO CAÑERO',
        how='left'
    )
    # Eliminar los registros donde no se encontró coincidencia (por ejemplo, no tienen nombre cañero)
    df_combinado = df_combinado[df_combinado['NOMBRE CAÑERO'].notna()]
    # Seleccionar columnas específicas
    df_combinado = df_combinado[['unidad_01', 'GRUPO DE COSECHA']].copy()
    # Convertir las columnas a enteros
    df_combinado['unidad_01'] = df_combinado['unidad_01'].astype(int)
    df_combinado['GRUPO DE COSECHA'] = df_combinado['GRUPO DE COSECHA'].astype(int)
    # Crear columna 'idd' con el formato unidad_01_GRUPODECOSECHA
    df_combinado['idd'] = df_combinado['unidad_01'].astype(str) + '_' + df_combinado['GRUPO DE COSECHA'].astype(str)
    # Eliminar duplicados según la columna 'idd'
    df_combinado = df_combinado.drop_duplicates(subset='idd')
    # Crear una lista para guardar los resultados de cada grupo
    relaciones = []
    # Obtener todos los grupos únicos
    grupos = sorted(df_combinado['GRUPO DE COSECHA'].unique())
    for grupo in grupos:
        # Filtrar cañeros y propiedades del grupo actual
        df_cañeros = df_grupos[df_grupos['GRUPO DE COSECHA'] == grupo][['CODIGO CAÑERO']].drop_duplicates()
        df_props = df_combinado[df_combinado['GRUPO DE COSECHA'] == grupo][['unidad_01']].drop_duplicates()
        # Si hay cañeros y propiedades, hacer el producto cartesiano
        if not df_cañeros.empty and not df_props.empty:
            df_temp = df_cañeros.merge(df_props, how='cross')
            df_temp['GRUPO DE COSECHA'] = grupo
            relaciones.append(df_temp)
    # Unir todos los grupos en un solo DataFrame
    df_relacion_total = pd.concat(relaciones, ignore_index=True)
    # Eliminar la columna de grupo
    df_relacion_total = df_relacion_total.drop(columns=['GRUPO DE COSECHA'])
    # Renombrar las columnas
    df_relacion_total = df_relacion_total.rename(columns={
        'CODIGO CAÑERO': 'COD_CANERO',
        'unidad_01': 'COD_PROPIE'
    })
    # Asegurarte de tener definida la ruta de salida
    archivo_salida = os.path.join(output_path, 'CANERO_PROPIEDAD.xlsx')
    # Exportar a Excel
    df_relacion_total.to_excel(archivo_salida, index=False)
    print('Se exporto CANERO_PROPIEDAD.xlsx exitosamente')

In [185]:
def crear_xlsx_lote_siembra_variedad():
    gdf_catastro = gpd.read_file(path_catastro)
    gdf_cat_filtrado = gdf_catastro[
        (gdf_catastro['obs'] == 'ok') &
        (gdf_catastro['cultivo'].isin(['canha', 'siembra']))]
    print('Area Total de Catastro:', gdf_cat_filtrado['area'].sum(), 'ha. para relacaion LOTE-SIEMBRA-CANA')
    df_datos = gdf_cat_filtrado[['unidad_01', 'unidad_03', 'unidad_05', 'area', 'variedad']]
    df_datos_orden = df_datos.sort_values(by=['unidad_01', 'unidad_05'])
    df_datos_orden['COD_LOTE'] = (
        df_datos_orden.groupby('unidad_01').cumcount() + 1
    )
    df_datos_orden.head(60)
    df_variedades = pd.read_excel(path_variedades)
    # Merge con indicador para identificar las filas sin coincidencia
    df_mergeado = df_datos_orden.merge(
        df_variedades[['COD_CANA', 'VARIEDAD', 'CATASTRO']],
        left_on='variedad',
        right_on='CATASTRO',
        how='left',
        indicator=True # crea un campo _merge que indica si tiene relacion o no
    )
    df_sin_relacion = df_mergeado[df_mergeado['_merge'] == 'left_only']
    df_con_relacion = df_mergeado[df_mergeado['_merge'] == 'both']
    df_con_relacion = df_con_relacion.copy()
    df_con_relacion.drop(columns=['_merge'], inplace=True)
    df_con_relacion.rename(columns={
        'unidad_01': 'COD_PROPIE',
        'unidad_03': 'COD_CANERO',
        'unidad_05': 'NOMBRE_LOT',
        'area': 'AREA'}, inplace=True)
    df_con_relacion['COD_SIEMBR'] = range(1, len(df_con_relacion) + 1)
    lote = df_con_relacion[['COD_LOTE', 'COD_PROPIE', 'NOMBRE_LOT', 'AREA']]
    siembra = df_con_relacion[['COD_SIEMBR', 'COD_CANERO', 'COD_PROPIE', 'COD_LOTE', 'COD_CANA']]
    cana = df_con_relacion[['COD_CANA', 'VARIEDAD']].drop_duplicates(subset='COD_CANA')
    cana = cana.sort_values(by='COD_CANA')
    # Asegurarte de tener definida la ruta de salida
    archivo_salida = os.path.join(output_path, 'LOTE.xlsx')
    # Exportar a Excel
    lote.to_excel(archivo_salida, index=False)
    # Asegurarte de tener definida la ruta de salida
    archivo_salida = os.path.join(output_path, 'SIEMBRA.xlsx')
    # Exportar a Excel
    siembra.to_excel(archivo_salida, index=False)
    # Asegurarte de tener definida la ruta de salida
    archivo_salida = os.path.join(output_path, 'CANA.xlsx')
    # Exportar a Excel
    cana.to_excel(archivo_salida, index=False)

    print('Area total procesada:', lote['AREA'].sum())
    print('Area con error variedad:', df_sin_relacion['area'].sum())
    print('Se creo LOTE.xlsx, SIEMBRA.xlsx, CANA.xlsx')

In [184]:
crear_xlsx_caneros()
crear_xlsx_propiedad()
crear_xlsx_caneho_propiedad()
crear_xlsx_lote_siembra_variedad()

1377 Cañeros registrados
Se exporto CANHEROS.xlsx exitosamente
Area Total de Catastro: 56139.12697181613 ha. para PROPIEDAD.xlsx
Cantidad de repetidos: 0
Se exporto PROPIEDAD.xlsx exitosamente
Area Total de Catastro: 56139.12697181613 ha. para relacion CANERO-PROPIEDAD
Se exporto CANERO_PROPIEDAD.xlsx exitosamente
Area Total de Catastro: 56139.12697181613 ha. para relacion CANERO-PROPIEDAD
Area total procesada: 56085.46530130274
Area con error variedad: 53.66167051338297
Se creo LOTE.xlsx, SIEMBRA.xlsx, CANA.xlsx


In [189]:
gdf_catastro = gpd.read_file(path_catastro)
gdf_cat_filtrado = gdf_catastro[
    (gdf_catastro['obs'] == 'ok') &
    (gdf_catastro['cultivo'].isin(['canha', 'siembra']))]
print('Area Total de Catastro:', gdf_cat_filtrado['area'].sum(), 'ha. para relacaion LOTE-SIEMBRA-CANA')
gdf_catastro

Area Total de Catastro: 56139.12697181613 ha. para relacaion LOTE-SIEMBRA-CANA


,fid,id,unidad_01,unidad_02,unidad_03,unidad_04,unidad_05,variedad,soca,zona,...,area,fc,obs,codco,nuevos,datosf,mesc,edad,coducaf,geometry
0,483.0,00002870000L2.1,287.0,ARCO IRIS NORTE III--MONASTERIO,8829.0,MONASTERIO FOIANINI SYLVIA D.,L2.1,RENOVACION,-1.0,69,...,9.891023,2024-06-18,ok,39.0,None,None,None,NaN,1051.0,"POLYGON ((473961.095 8075944.601, 473970.716 8..."
1,496.0,00003760000L2,376.0,NARANJAL--AGUILERA SANTIAGO,18027.0,AGUILERA TARRADELLES GUILLERMO,L2,RENOVACION,-1.0,50,...,10.722511,2024-08-14,ok,198.0,None,None,None,NaN,267.0,"POLYGON ((480489.148 8077720.927, 480463.927 8..."
2,505.0,00003760000L3,376.0,NARANJAL--AGUILERA SANTIAGO,18027.0,AGUILERA TARRADELLES GUILLERMO,L3,RENOVACION,-1.0,50,...,12.056506,2024-08-02,ok,198.0,None,None,None,NaN,267.0,"POLYGON ((480339.263 8078320.173, 480618.195 8..."
3,510.0,00003240000L11,324.0,EL NARANJAL--GUTIERREZ ALFREDO,4839.0,GUTIERREZ TARRADELLES ALFREDO,L11,RENOVACION,-1.0,50,...,6.341586,2024-08-26,ok,52.0,None,None,None,NaN,287.0,"POLYGON ((480073.135 8080133.347, 480076.971 8..."
4,511.0,00003760000L1,376.0,NARANJAL--AGUILERA SANTIAGO,18027.0,AGUILERA TARRADELLES GUILLERMO,L1,RENOVACION,-1.0,50,...,7.616127,2024-08-14,ok,198.0,None,None,None,NaN,267.0,"POLYGON ((480134.811 8077634.423, 480154.729 8..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13454,26284.0,00006770000L1.1,677.0,EVANGELISTA--TENORIO,42237.0,TENORIO LEONOR CERMENO RODAS DE,L1.1,RENOVACION,NaN,81,...,1.201410,None,ok,300.0,None,None,None,NaN,1127.0,"POLYGON ((490264.157 8097736.732, 490279.874 8..."
13455,26285.0,None,2025.0,None,8023.0,None,None,UCG9020,0.0,None,...,1.653056,None,ok,NaN,None,None,None,NaN,NaN,"POLYGON ((464176.706 8095036.115, 464336.752 8..."
13456,26286.0,None,986.0,None,13003.0,None,None,UCG9020,5.0,None,...,3.868736,None,ok,NaN,None,None,None,NaN,NaN,"POLYGON ((491078.280 8093100.068, 491208.652 8..."
13457,26293.0,None,986.0,None,13003.0,None,None,RBB7726,5.0,None,...,3.427438,None,ok,NaN,None,None,None,NaN,NaN,"POLYGON ((490879.542 8093992.029, 490891.885 8..."


In [188]:
gdf_catastro

,fid,id,unidad_01,unidad_02,unidad_03,unidad_04,unidad_05,variedad,soca,zona,...,area,fc,obs,codco,nuevos,datosf,mesc,edad,coducaf,geometry
0,483.0,00002870000L2.1,287.0,ARCO IRIS NORTE III--MONASTERIO,8829.0,MONASTERIO FOIANINI SYLVIA D.,L2.1,RENOVACION,-1.0,69,...,9.891023,2024-06-18,ok,39.0,None,None,None,NaN,1051.0,"POLYGON ((473961.095 8075944.601, 473970.716 8..."
1,496.0,00003760000L2,376.0,NARANJAL--AGUILERA SANTIAGO,18027.0,AGUILERA TARRADELLES GUILLERMO,L2,RENOVACION,-1.0,50,...,10.722511,2024-08-14,ok,198.0,None,None,None,NaN,267.0,"POLYGON ((480489.148 8077720.927, 480463.927 8..."
2,505.0,00003760000L3,376.0,NARANJAL--AGUILERA SANTIAGO,18027.0,AGUILERA TARRADELLES GUILLERMO,L3,RENOVACION,-1.0,50,...,12.056506,2024-08-02,ok,198.0,None,None,None,NaN,267.0,"POLYGON ((480339.263 8078320.173, 480618.195 8..."
3,510.0,00003240000L11,324.0,EL NARANJAL--GUTIERREZ ALFREDO,4839.0,GUTIERREZ TARRADELLES ALFREDO,L11,RENOVACION,-1.0,50,...,6.341586,2024-08-26,ok,52.0,None,None,None,NaN,287.0,"POLYGON ((480073.135 8080133.347, 480076.971 8..."
4,511.0,00003760000L1,376.0,NARANJAL--AGUILERA SANTIAGO,18027.0,AGUILERA TARRADELLES GUILLERMO,L1,RENOVACION,-1.0,50,...,7.616127,2024-08-14,ok,198.0,None,None,None,NaN,267.0,"POLYGON ((480134.811 8077634.423, 480154.729 8..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13454,26284.0,00006770000L1.1,677.0,EVANGELISTA--TENORIO,42237.0,TENORIO LEONOR CERMENO RODAS DE,L1.1,RENOVACION,NaN,81,...,1.201410,None,ok,300.0,None,None,None,NaN,1127.0,"POLYGON ((490264.157 8097736.732, 490279.874 8..."
13455,26285.0,None,2025.0,None,8023.0,None,None,UCG9020,0.0,None,...,1.653056,None,ok,NaN,None,None,None,NaN,NaN,"POLYGON ((464176.706 8095036.115, 464336.752 8..."
13456,26286.0,None,986.0,None,13003.0,None,None,UCG9020,5.0,None,...,3.868736,None,ok,NaN,None,None,None,NaN,NaN,"POLYGON ((491078.280 8093100.068, 491208.652 8..."
13457,26293.0,None,986.0,None,13003.0,None,None,RBB7726,5.0,None,...,3.427438,None,ok,NaN,None,None,None,NaN,NaN,"POLYGON ((490879.542 8093992.029, 490891.885 8..."
